# AR(1)プロセスからランダムウォーク、そしてMCMCへ

このノートブックでは、時系列解析の基礎となる **AR(1)プロセス**（1次自己回帰モデル）を出発点とし、その係数 $\phi$ が 1 に近づくにつれて現れる **ランダムウォーク（酔歩）**、そしてそのランダムウォークを「探索」に応用した **メトロポリス・ヘイスティングス法 (MCMC)** までの流れを、数式とインタラクティブなシミュレーションで地続きに解説します。

---

## 1. AR(1) プロセスとは

AR(1) プロセス（AutoRegressive process of order 1）は、**「現在の値が、1つ前の値とノイズで決まる」** 最もシンプルな時系列モデルです。

$$
y_t = \phi y_{t-1} + \varepsilon_t, \quad \varepsilon_t \sim N(0, \sigma^2)
$$

ここで、
- $y_t$: 時刻 $t$ における値
- $\phi$: **自己回帰係数** (Autoregressive coefficient)
- $\varepsilon_t$: ホワイトノイズ（平均0、分散$\sigma^2$の正規分布に従う）

この $\phi$ の値によって、時系列の挙動（**定常性**）が劇的に変化します。

- **$|\phi| < 1$ (定常)**: 平均 0 の周りを振動し、長期的には安定します。
- **$|\phi| = 1$ (単位根・ランダムウォーク)**: **非定常**。平均へ回帰せず、どこまでも彷徨います。
- **$|\phi| > 1$ (発散)**: 指数関数的に値が大きくなります。

### 自己相関 (ACF) と 偏自己相関 (PACF)
ARモデルの特徴は、**コレログラム**（自己相関図）に現れます。

- **自己相関関数 (ACF)**: 時間差（ラグ）$k$ だけ離れたデータ間の相関。
    - AR(1) の場合、ACFは $\phi^k$ で減衰します（$\phi=0.8$なら $0.8, 0.64, 0.512...$）。
- **偏自己相関関数 (PACF)**: 間の時点の影響を取り除いた直接的な相関。
    - AR(1) の場合、**ラグ1 ( $\phi$ ) だけが有意な値を持ち、ラグ2以降はゼロになります**（これがARモデルの識別基準です）。

---


### 【実験】 $\phi$ を変えて挙動を確認しよう

スライダーで $\phi$ を動かして、以下の点を確認してください。
1. **$\phi = 0.5$**: ACFは速やかに減衰し、PACFはラグ1のみ立つ。
2. **$\phi = 0.9$**: ACFの減衰が遅くなる（記憶が長く残る）。
3. **$\phi = 1.0$**: **ランダムウォーク**。ACFはほとんど減衰せず、1付近に留まる（これを「単位根」と呼ぶ）。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ipywidgets as widgets
from IPython.display import display
import japanize_matplotlib

def simulate_ar1_interactive(phi=0.8, sigma=1.0, T=200):
    # シミュレーション
    np.random.seed(42)
    y = np.zeros(T)
    epsilon = np.random.normal(0, sigma, T)
    
    # 初期値
    if abs(phi) < 1:
        y[0] = epsilon[0] / np.sqrt(1 - phi**2)
    else:
        y[0] = 0
        
    for t in range(1, T):
        y[t] = phi * y[t-1] + epsilon[t]
        
    # プロット作成
    fig = plt.figure(figsize=(15, 5))
    
    # 1. 時系列プロット
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.plot(y)
    ax1.set_title(rf"時系列 $y_t = {phi} y_{{t-1}} + \varepsilon$")
    ax1.set_xlabel("時刻 $t$")
    ax1.set_ylabel("$y_t$")
    ax1.grid(True, alpha=0.3)
    if abs(phi) >= 1:
        ax1.text(0.05, 0.9, "非定常 (Non-Stationary)", transform=ax1.transAxes, color="red", fontweight="bold")
    else:
        ax1.text(0.05, 0.9, "定常 (Stationary)", transform=ax1.transAxes, color="blue", fontweight="bold")

    # 2. ACF (自己相関)
    ax2 = fig.add_subplot(1, 3, 2)
    plot_acf(y, lags=20, ax=ax2, title="自己相関 (ACF)")
    ax2.grid(True, alpha=0.3)
    
    # 3. PACF (偏自己相関)
    ax3 = fig.add_subplot(1, 3, 3)
    try:
        plot_pacf(y, lags=20, ax=ax3, title="偏自己相関 (PACF)", method='ywm')
    except:
        ax3.text(0.5, 0.5, "計算エラー (非定常)", ha='center')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# インタラクティブ・ウィジェット
w_phi = widgets.FloatSlider(value=0.8, min=-1.1, max=1.1, step=0.1, description=r'係数 $\phi$:', continuous_update=False)
display(widgets.interactive(simulate_ar1_interactive, phi=w_phi, sigma=widgets.fixed(1.0), T=widgets.fixed(200)))


---

## 2. 定常から非定常へ：ランダムウォーク（酔歩）

$\phi = 1$ のとき、式は以下のようになります。

$$
y_t = y_{t-1} + \varepsilon_t
$$

これは、**「昨日の位置からランダムに一歩進んだ場所が今日の位置」** というモデルです。これを **ランダムウォーク（酔歩）** と呼びます。またの名を「イカ（Squid）」の遊泳とも喩えられます（予測不可能な動き）。

### なぜこれが重要なのか？
ランダムウォークは、**「分散が時間とともに増大する」** という性質を持ちます。
$$
\mathrm{Var}(y_t) = t \sigma^2
$$
つまり、時間が経てば経つほど、**どこにいるか予測がつかなくなる（＝探索範囲が広がる）** ということです。

統計学（特にベイズ統計）では、この「どこへ行くかわからない＝空間を広く探索できる」という性質を**逆に利用**します。それが次に紹介する **MCMC（マルコフ連鎖モンテカルロ法）** です。


---

## 3. ランダムウォークの応用：メトロポリス・ヘイスティングス法

確率分布 $p(x)$ からサンプリングを行いたいとき、その分布が複雑だと直接サンプリングできません。そこで、**ランダムウォークを使って分布の「山」を探索**します。

### アルゴリズムの直感
1. **今いる場所** を $x_t$ とします。
2. 次の候補 $x_{new}$ を、**ランダムウォーク** で決めます。
   $$ x_{new} = x_t + \varepsilon, \quad \varepsilon \sim N(0, \sigma^2) $$
   （これはまさに、$\phi=1$ の AR(1) プロセスの1ステップです！）
3. $x_{new}$ が $x_t$ よりも「確率が高い（山頂に近い）」場所なら、必ず移動します。
4. 「確率が低い（麓の方）」場所なら、コイン投げをして、運が良ければ移動、悪ければ留まります。

こうすることで、ランダムウォーク（酔っ払い）は、**確率が高い場所ほど頻繁に訪れる**ようになります。その足跡（Trace）を集めると、目的の確率分布に従うサンプルが得られます。



### 目標分布の可視化
まずはサンプリングしたい **目標分布** $\pi(x)$ を図で確認します。


In [ ]:
from scipy.stats import norm

# 目標分布: 標準正規分布 N(0, 1)
def target_pdf(x):
    return norm.pdf(x, loc=0, scale=1)

x = np.linspace(-4, 4, 400)
plt.figure(figsize=(7, 4))
plt.plot(x, target_pdf(x), color='crimson', lw=3, label=r'$\pi(x)$（目標分布）')
plt.title('目標分布 $\pi(x)$', fontsize=13)
plt.xlabel('x')
plt.ylabel('密度')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 提案分布の可視化（ランダムウォーク）
メトロポリス・ヘイスティングス法では、現在位置 $x_t$ から
$$x_{new} = x_t + ε,\quad ε \sim N(0, σ^2)$$
という **提案分布** を使います。ここでは $x_t=1.5$ と仮定して可視化します。


In [ ]:
current_x = 1.5
step_width = 1.0

proposal_x = np.linspace(-4, 6, 400)
proposal_pdf = norm.pdf(proposal_x, loc=current_x, scale=step_width)

plt.figure(figsize=(7, 4))
plt.plot(proposal_x, proposal_pdf, color='navy', lw=3, label='提案分布 $q(x\'|x_t)$')
plt.axvline(current_x, color='orange', ls='--', lw=2, label=r'現在位置 $x_t$')
plt.title('提案分布（ランダムウォーク）', fontsize=13)
plt.xlabel('x')
plt.ylabel('密度')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 採択確率の形（途中計算の可視化）
採択確率は
$$
\alpha(x_{new}, x_t) = \min\left(1, \frac{\pi(x_{new})}{\pi(x_t)}\right)
$$
で与えられます。$x_t$ を固定したとき、$x_{new}$ に対する形を描いてみます。


In [ ]:
current_x = 1.5
x_new = np.linspace(-4, 4, 400)
ratio = target_pdf(x_new) / target_pdf(current_x)
accept_prob = np.minimum(1.0, ratio)

plt.figure(figsize=(7, 4))
plt.plot(x_new, accept_prob, color='purple', lw=3)
plt.axvline(current_x, color='orange', ls='--', lw=2, label=r'現在位置 $x_t$')
plt.title('採択確率 $\alpha(x_{new}, x_t)$', fontsize=13)
plt.xlabel('提案値 $x_{new}$')
plt.ylabel('採択確率')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

def run_metropolis_hastings(n_iter=5000, step_width=1.0):
    # 目標分布: 標準正規分布 N(0, 1) とあえて少しズラした N(3, 0.5) の混合分布を想定してみる
    # シンプルにするため、ここでは標準正規分布 N(0, 1) を目標とする
    np.random.seed(123)
    current_x = 10.0  # 初期値（あえて遠くからスタート）
    samples = []
    trace = []
    
    # 提案分布としてランダムウォークを使用
    # x_new = 1.0 * x_old + epsilon  (つまり phi=1 の AR(1))
    
    for i in range(n_iter):
        # 1. 提案 (Random Walk Step)
        epsilon = np.random.normal(0, step_width)
        proposal_x = current_x + epsilon
        
        # 2. 採択確率 (Metropolis基準)
        # p(x_new) / p(x_old)
        ratio = target_pdf(proposal_x) / target_pdf(current_x)
        acceptance_prob = min(1.0, ratio)
        
        # 3. 判定
        if np.random.rand() < acceptance_prob:
            current_x = proposal_x  # 採択（移動）
        else:
            current_x = current_x   # 棄却（滞留）
            
        trace.append(current_x)
        if i > 500: # バーンイン（初期の影響を除く）
            samples.append(current_x)
            
    # 可視化
    fig = plt.figure(figsize=(15, 6))
    
    # トレースプロット（時系列）
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(trace, lw=0.8, color='navy', alpha=0.7)
    # 生文字列でエスケープを回避
    ax1.set_title(rf"MCMCの軌跡 (トレースプロット)\nランダムウォーク $\phi=1$ による探索", fontsize=14)
    ax1.set_xlabel("Step $t$")
    ax1.set_ylabel("$x_t$")
    ax1.grid(True, alpha=0.3)
    ax1.text(0.05, 0.9, "探索的挙動 (探索)", transform=ax1.transAxes, color="green", fontweight="bold")

    # ヒストグラム（分布）
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.hist(samples, bins=50, density=True, color='skyblue', alpha=0.7, label="MCMCサンプル")
    x = np.linspace(-4, 4, 100)
    ax2.plot(x, target_pdf(x), 'r-', lw=3, label="目標分布")
    ax2.set_title("サンプリング結果の分布", fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 実行
run_metropolis_hastings()



---
## （付録）平均の分散に関する理論的考察

まず平均を
$$
\bar{y}_T = \frac{1}{T}\sum_{t=1}^T y_t
$$
と置くと、自己共分散 $\gamma_h=\mathrm{Cov}(y_t,y_{t-h})$ を用いて
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\sum_{t=1}^T\sum_{s=1}^T\gamma_{|t-s|}
$$
となります。AR(1)では $\gamma_h=\sigma_u^2\phi^{|h|}$ なので、二重和を整理すると
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\Big(T\gamma_0 + 2\sum_{h=1}^{T-1}(T-h)\gamma_h\Big)
$$
$$
= \frac{1}{T^2}\Big(T\sigma_u^2 + 2\sum_{h=1}^{T-1}(T-h)\sigma_u^2\phi^h\Big)
$$
となり、相関がある（$\phi \neq 0$）と、独立な場合（iid）に比べて分散が変化することがわかります。

---
